# 导师制名企实训班商业智能方向 004期 Lesson 6

### Thinking 1: 什么是近似最近邻查找，常用的方法有哪些

近似最近邻检索是在牺牲可接受范围内的精度的情况下提高检索效果，最近邻检索是线性复杂度的，当处理大规模数据时有优势。  
有两种比较流行的方法：树方法和哈希方法  

<b>精确检索：</b>基于树结构的最近邻检索方法
索引树就是最常见的方法。其基本思想是对搜索空间进行层次划分，再进行快速匹配。当数据维度不太高（如d< 20)，通常采用树型索引结构对数据进行分区以实现高效索引，如最经典的KD树算法 、R树、M树等等，它们的时间和空间复杂度都是以d为指数的指数级别的，在实际搜索时也取得了良好的效果。
* 基于树的方法的一些特点：  
1. 递归了划分数据：分而治之。  
2. 查询时间为：$O(log n) $
3. 随着数据维数的增加，基于树的ANN其表现性能会急剧的下降。  
4. 需要的存储开销很大，因为需要存储树结构。  
5. 在运行的时候，需要保存原始数据。同样会增加内存的开销。  

<b>近似检索：</b>基于哈希散列的方法和基于矢量量化方法  
搜索可能是近邻的数据项而不再只局限于返回最可能的项目，在牺牲可接受范围内的精度的情况下提高检索效率。
* 哈希方法的一些特点：
1. 数据库中的每一个item都被用一个编码来表达。
2. 可以极大的降低内存空间。  
3. 查询时间为：$ O(1) $或是线性的。  

哈希方法包括：局部敏感哈希Locality-Sensitive Hashing（LSH）方法：在高维空间相邻的数据经过哈希函数的映射投影转化到低维空间后，他们落入同一个吊桶的概率很大而不相邻的数据映射到同一个吊桶的概率则很小。在检索时将欧式空间的距离计算转化到汉明（Hamming）空间，并将全局检索转化为对映射到同一个吊桶中的数据进行检索，从而提高了检索速度。这种方法的主要难点在于如何寻找适合的哈希函数。  
矢量量化方法：其代表是乘积量化（PQ）：将特征向量进行正交分解，在分解后的低维正交子空间上进行量化，由于低维空间可以采用较小的码本进行编码，因此可以降低数据存储空间 。

### Thinking 2: 为什么两个集合的minhash值相同的概率等于这两个集合的Jaccard相似度

两个文档$i$和B$j$的MinHash值相同的概率  
A = 两列值都为1的个数  
B = 其中一列值为1的个数  
$$P(h(C_i)=h(C_j))=\frac{A}{A+B}$$   
而Jaccard相似度也等于  
$$\frac{A}{A+B}$$

### Thinking 3： SimHash在计算文档相似度的作用是怎样的？

SimHash算法高效，适用于长文本和数据量大的数据集。SimHash在计算文档相似度的作用是生成文档的指纹，相当于提取了文档的特征。一篇文章通过SimHash算法计算出其指纹，之后可以通过计算Hamming距离来计算不同文档之间的相似度。

### Thinking 4： 为什么YouTube采用期望观看时间作为评估指标

YouTube是个视频网站，在YouTube上如果只使用是否点击视频为评估指标是不准确的，可能存在有些视频点开看了很短时间发现不是自己喜欢的食品就关闭了，所以采用期望观看时间作为评价指标更为合理。此外，个人认为有另外一个原因，YouTube上的很多视频中是内嵌广告的，而且广告在视频内的时间点是不同的，这样的话只有推荐给用户的视频是期望观看时间长的视频才能达到尽可能大的收益。

### Action 1： 使用MinHashLSHForest对微博新闻句子进行检索
weibo.txt  
针对某句话进行Query，查找Top-3相似的句子

In [1]:
# 对weibo.txt进行相似句子Top-K查询
from datasketch import MinHash, MinHashLSHForest
import jieba.posseg as pseg
import re

In [2]:
# 读取文件
f = open('data/weibos.txt', 'r', encoding='UTF-8')
text = f.read()
text

'#斯科拉里愿意执教国足#上一届如果是里皮从头芾到尾，是很大机会入世界杯的，这一届，没几个能用的，除非大力归化，谁来都没用。 \u200b\n国足输给叙利亚之后，里皮辞职。谁将成为新主帅，成为广大球迷关注的焦点。目前舆论方面，倾向于三个人：山东鲁能主帅李霄鹏、武汉卓尔主帅李铁、前广州恒大主帅斯科拉里。 \u200b\n据了解，无论中国足协态度如何，里皮其实在宣布请辞同时已经去意已决。据了解。比赛当晚，他的太太西蒙内塔女士及儿子小里皮都在现场看台上观战。辞职后的里皮没有改变原有的计划——赛后第二天他会从迪拜直接飞回意大利。这意味着，他本来也没打算与球队管理层或中国足协高层在赛后第一时间内进行有关辞职的对话。至于辞职以后的善后工作包括合同问题的沟通工作也要待日后双方进一步协商。\n让我们回顾一下国足历届外籍教练——里皮，佩兰，卡马乔，杜伊科维奇，阿里·汉，米卢……。来之前一个比一个有名，来之后一个比一个水，国足踢不好完全是足协的问题，足协不解散重组，你把天王老子请来都不行\n斯科拉里想执教中国国足！老头有点意思，凡是里皮干过的地方，他就想试试。当然，老头也是世界杯冠军教头，万一折在中国这里也没啥丢人的，毕竟里皮也折了嘛！可以试试！\n斯科拉里的水平，还不如里皮。斯科拉里，看好的不是国足，而是年薪…… \u200b\n非常应该辞职！中国足球，不需要名帅，也不需要外籍教练，因为一点儿毛用也没有！从施拉普纳到现在，二十余年间，中国足球竟然大踏步的倒退，一点儿也杀不住车，奶奶的，刹车系统坏了！穿着几百块钱的球衣，几千块钱的球鞋，几万块钱的包，几十万的包机，几百万上千万的年薪\n赛后，叙利亚主教练在更衣室里给每个队员一个耳光。主教练说：“赛前老子就再三交代，这一场无论如何都不能赢中国队！中国援助了我们那么多粮食和美金，如果他们不再援助我们国家，你狗日些要吃土去！”，球员委屈的说：“七十多分钟了，哪个晓得那个龟儿子往他们家球门踢嘛？”\n里皮辞职返回意大利，他的助教马达洛尼随队返回广州。马达洛尼在接受采访时还原了当时更衣室中的情况：“当时在更衣室，球员们都过来试图说服里皮，让他收回决定，队长郑智尝试阻止他，足协的代表也希望他在考虑一下，我也建议他重新考虑，但无济于事。”\n中国足协：接受里皮辞职请求，将深刻反思\n看了个报道，马达洛尼说：“关于里皮的辞职，我事先也没有被告知，自己也

In [3]:
# 以句号，叹号，问号作为分隔，去掉\n换行符号,#号，\u200b和前后空格
sentences = re.split('[。！？…]', text.replace('\n', '').replace('#','').replace('\u200b',''))
sentences

['斯科拉里愿意执教国足上一届如果是里皮从头芾到尾，是很大机会入世界杯的，这一届，没几个能用的，除非大力归化，谁来都没用',
 ' 国足输给叙利亚之后，里皮辞职',
 '谁将成为新主帅，成为广大球迷关注的焦点',
 '目前舆论方面，倾向于三个人：山东鲁能主帅李霄鹏、武汉卓尔主帅李铁、前广州恒大主帅斯科拉里',
 ' 据了解，无论中国足协态度如何，里皮其实在宣布请辞同时已经去意已决',
 '据了解',
 '比赛当晚，他的太太西蒙内塔女士及儿子小里皮都在现场看台上观战',
 '辞职后的里皮没有改变原有的计划——赛后第二天他会从迪拜直接飞回意大利',
 '这意味着，他本来也没打算与球队管理层或中国足协高层在赛后第一时间内进行有关辞职的对话',
 '至于辞职以后的善后工作包括合同问题的沟通工作也要待日后双方进一步协商',
 '让我们回顾一下国足历届外籍教练——里皮，佩兰，卡马乔，杜伊科维奇，阿里·汉，米卢',
 '',
 '',
 '来之前一个比一个有名，来之后一个比一个水，国足踢不好完全是足协的问题，足协不解散重组，你把天王老子请来都不行斯科拉里想执教中国国足',
 '老头有点意思，凡是里皮干过的地方，他就想试试',
 '当然，老头也是世界杯冠军教头，万一折在中国这里也没啥丢人的，毕竟里皮也折了嘛',
 '可以试试',
 '斯科拉里的水平，还不如里皮',
 '斯科拉里，看好的不是国足，而是年薪',
 '',
 ' 非常应该辞职',
 '中国足球，不需要名帅，也不需要外籍教练，因为一点儿毛用也没有',
 '从施拉普纳到现在，二十余年间，中国足球竟然大踏步的倒退，一点儿也杀不住车，奶奶的，刹车系统坏了',
 '穿着几百块钱的球衣，几千块钱的球鞋，几万块钱的包，几十万的包机，几百万上千万的年薪赛后，叙利亚主教练在更衣室里给每个队员一个耳光',
 '主教练说：“赛前老子就再三交代，这一场无论如何都不能赢中国队',
 '中国援助了我们那么多粮食和美金，如果他们不再援助我们国家，你狗日些要吃土去',
 '”，球员委屈的说：“七十多分钟了，哪个晓得那个龟儿子往他们家球门踢嘛',
 '”里皮辞职返回意大利，他的助教马达洛尼随队返回广州',
 '马达洛尼在接受采访时还原了当时更衣室中的情况：“当时在更衣室，球员们都过来试图说服里皮，让他收回决定，队长郑智尝试阻止他，足协的代表也希望他在考虑

In [4]:
# 删除空行和前后空格和其他符号
for i in range(len(sentences)):
    sentences[i] = sentences[i].strip('”， ')
for sentence in sentences:
    if sentence == '':
        sentences.remove(sentence)
# 最后一行如果为空，则删除
if sentences[len(sentences)-1] == '':
    sentences.pop()
sentences

['斯科拉里愿意执教国足上一届如果是里皮从头芾到尾，是很大机会入世界杯的，这一届，没几个能用的，除非大力归化，谁来都没用',
 '国足输给叙利亚之后，里皮辞职',
 '谁将成为新主帅，成为广大球迷关注的焦点',
 '目前舆论方面，倾向于三个人：山东鲁能主帅李霄鹏、武汉卓尔主帅李铁、前广州恒大主帅斯科拉里',
 '据了解，无论中国足协态度如何，里皮其实在宣布请辞同时已经去意已决',
 '据了解',
 '比赛当晚，他的太太西蒙内塔女士及儿子小里皮都在现场看台上观战',
 '辞职后的里皮没有改变原有的计划——赛后第二天他会从迪拜直接飞回意大利',
 '这意味着，他本来也没打算与球队管理层或中国足协高层在赛后第一时间内进行有关辞职的对话',
 '至于辞职以后的善后工作包括合同问题的沟通工作也要待日后双方进一步协商',
 '让我们回顾一下国足历届外籍教练——里皮，佩兰，卡马乔，杜伊科维奇，阿里·汉，米卢',
 '来之前一个比一个有名，来之后一个比一个水，国足踢不好完全是足协的问题，足协不解散重组，你把天王老子请来都不行斯科拉里想执教中国国足',
 '老头有点意思，凡是里皮干过的地方，他就想试试',
 '当然，老头也是世界杯冠军教头，万一折在中国这里也没啥丢人的，毕竟里皮也折了嘛',
 '可以试试',
 '斯科拉里的水平，还不如里皮',
 '斯科拉里，看好的不是国足，而是年薪',
 '非常应该辞职',
 '中国足球，不需要名帅，也不需要外籍教练，因为一点儿毛用也没有',
 '从施拉普纳到现在，二十余年间，中国足球竟然大踏步的倒退，一点儿也杀不住车，奶奶的，刹车系统坏了',
 '穿着几百块钱的球衣，几千块钱的球鞋，几万块钱的包，几十万的包机，几百万上千万的年薪赛后，叙利亚主教练在更衣室里给每个队员一个耳光',
 '主教练说：“赛前老子就再三交代，这一场无论如何都不能赢中国队',
 '中国援助了我们那么多粮食和美金，如果他们不再援助我们国家，你狗日些要吃土去',
 '球员委屈的说：“七十多分钟了，哪个晓得那个龟儿子往他们家球门踢嘛',
 '里皮辞职返回意大利，他的助教马达洛尼随队返回广州',
 '马达洛尼在接受采访时还原了当时更衣室中的情况：“当时在更衣室，球员们都过来试图说服里皮，让他收回决定，队长郑智尝试阻止他，足协的代表也希望他在考虑一下，我也建议他重新考虑，但无济于事',


In [5]:
# 将item_text进行分词
def get_item_str(item_text):
    item_str = "" 
    item=(pseg.cut(item_text)) 
    for i in list(item):
        #去掉停用词
        if i.word not in list(stop):  
            item_str += i.word
            #tfidf_vectorizer.fit_transform的输入需要空格分隔的单词
            item_str += " "
    return item_str

In [6]:
# 对item_str创建MinHash
def get_minhash(item_str):
    temp = MinHash()
    for d in item_str:
        temp.update(d.encode('utf8'))
    return temp

In [7]:
# 设置停用词
stop = [line.strip().decode('utf-8') for line in open('data/stopword.txt').readlines()]
# 得到分词后的documents
documents = []
for item_text in sentences:
    # 将item_text进行分词
    item_str = get_item_str(item_text)
    documents.append(item_str)

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\LinWang\AppData\Local\Temp\jieba.cache
Loading model cost 0.556 seconds.
Prefix dict has been built succesfully.


In [8]:
# 创建LSH Forest及MinHash对象
minhash_list = []
forest = MinHashLSHForest()
for i in range(len(documents)):
    #得到train_documents[i]的MinHash
    temp = get_minhash(documents[i])
    minhash_list.append(temp)
    forest.add(i, temp)
# index所有key，以便可以进行检索
forest.index()

In [9]:
query = sentences[0]
query

'斯科拉里愿意执教国足上一届如果是里皮从头芾到尾，是很大机会入世界杯的，这一届，没几个能用的，除非大力归化，谁来都没用'

In [10]:
# 将item_text进行分词
item_str = get_item_str(query)
item_str

'斯科拉里 愿意 执教 国足 上 一届 如果 是 里 皮 从头 芾 到 尾 ， 是 很大 机会 入 世界杯 的 ， 这 一届 ， 没 几个 能用 的 ， 除非 大力 归化 ， 谁 来 都 没用 '

In [11]:
# 得到item_str的MinHash
minhash_query = get_minhash(item_str)

In [12]:
# 查询forest中与m1相似的Top-K个邻居
result = forest.query(minhash_query, 3)
for i in range(len(result)):
    print(result[i], minhash_query.jaccard(minhash_list[result[i]]), documents[result[i]].replace(' ', ''))
print("Top 3 邻居", result)

0 1.0 斯科拉里愿意执教国足上一届如果是里皮从头芾到尾，是很大机会入世界杯的，这一届，没几个能用的，除非大力归化，谁来都没用
11 0.25 来之前一个比一个有名，来之后一个比一个水，国足踢不好完全是足协的问题，足协不解散重组，你把天王老子请来都不行斯科拉里想执教中国国足
13 0.296875 当然，老头也是世界杯冠军教头，万一折在中国这里也没啥丢人的，毕竟里皮也折了嘛
Top 3 邻居 [0, 11, 13]


### 三国

In [13]:
# 读取文件
f1 = open('data/three_kingdoms.txt', 'r', encoding='UTF-8')
text1 = f1.read()

In [14]:
# 以句号，叹号，问号作为分隔，去掉\n换行符号,#号，\u200b和前后空格
sentences1 = re.split('[。！？…]', text1.replace('\n', '').replace('#','').replace('\u3000',''))
sentences1[:10]

['三国演义作者：罗贯中正文 第一回 宴桃园豪杰三结义 斩黄巾英雄首立功滚滚长江东逝水，浪花淘尽英雄',
 '是非成败转头空',
 '青山依旧在，几度夕阳红',
 '白发渔樵江渚上，惯看秋月春风',
 '一壶浊酒喜相逢',
 '古今多少事，都付笑谈中',
 '——调寄《临江仙》话说天下大势，分久必合，合久必分',
 '周末七国分争，并入于秦',
 '及秦灭之后，楚、汉分争，又并入于汉',
 '汉朝自高祖斩白蛇而起义，一统天下，后来光武中兴，传至献帝，遂分为三国']

In [15]:
# 删除空行和前后空格和其他符号
for i in range(len(sentences1)):
    sentences1[i] = sentences1[i].strip('”， -')
for sentence in sentences1:
    if sentence == '':
        sentences1.remove(sentence)
# 最后一行如果为空，则删除
if sentences1[len(sentences)-1] == '':
    sentences1.pop()
sentences1[:10]

['三国演义作者：罗贯中正文 第一回 宴桃园豪杰三结义 斩黄巾英雄首立功滚滚长江东逝水，浪花淘尽英雄',
 '是非成败转头空',
 '青山依旧在，几度夕阳红',
 '白发渔樵江渚上，惯看秋月春风',
 '一壶浊酒喜相逢',
 '古今多少事，都付笑谈中',
 '——调寄《临江仙》话说天下大势，分久必合，合久必分',
 '周末七国分争，并入于秦',
 '及秦灭之后，楚、汉分争，又并入于汉',
 '汉朝自高祖斩白蛇而起义，一统天下，后来光武中兴，传至献帝，遂分为三国']

In [16]:
# 将item_text进行分词
def get_item_str(item_text):
    item_str = "" 
    item=(pseg.cut(item_text)) 
    for i in list(item):
        #去掉停用词
        if i.word not in list(stop):  
            item_str += i.word
            #tfidf_vectorizer.fit_transform的输入需要空格分隔的单词
            item_str += " "
    return item_str

In [17]:
# 得到分词后的documents
documents1 = []
for item_text in sentences1:
    # 将item_text进行分词
    item_str1 = get_item_str(item_text)
    documents1.append(item_str1)

In [18]:
# 创建LSH Forest及MinHash对象
minhash_list1 = []
forest1 = MinHashLSHForest()
for i in range(len(documents1)):
    #得到train_documents[i]的MinHash
    temp1 = get_minhash(documents1[i])
    minhash_list1.append(temp1)
    forest1.add(i, temp1)
# index所有key，以便可以进行检索
forest1.index()

In [19]:
query1 = sentences1[0]
query1

'三国演义作者：罗贯中正文 第一回 宴桃园豪杰三结义 斩黄巾英雄首立功滚滚长江东逝水，浪花淘尽英雄'

In [20]:
# 将item_text进行分词
item_str1 = get_item_str(query1)
item_str1

'三国演义 作者 ： 罗贯中 正文   第一回   宴 桃园 豪杰 三 结义   斩 黄巾 英雄 首 立功 滚滚 长江 东 逝水 ， 浪花 淘尽 英雄 '

In [21]:
# 得到item_str的MinHash
minhash_query1 = get_minhash(item_str1)

In [22]:
# 查询forest中与m1相似的Top-K个邻居
result1 = forest1.query(minhash_query1, 3)
for i in range(len(result1)):
    print(result1[i], minhash_query1.jaccard(minhash_list1[result1[i]]), documents1[result1[i]].replace(' ', ''))
print("Top 3 邻居", result1)

0 1.0 三国演义作者：罗贯中正文第一回宴桃园豪杰三结义斩黄巾英雄首立功滚滚长江东逝水，浪花淘尽英雄
339 0.109375 玄德曰：“备乃中山靖王之后；自涿郡剿戮黄巾，大小三十余战，颇有微功，因得除今职
13299 0.09375 云长见一老将出马，知是黄忠，把五百校刀手一字摆开，横刀立马而问曰：“来将莫非黄忠否
Top 3 邻居 [0, 339, 13299]
